In [1]:
import modif_bert
import transformers_modif
from torch import nn, optim
import torch.nn.functional as F
import torch
import numpy as np
import spacy

I1118 16:43:33.145685 140578894108480 file_utils.py:39] PyTorch version 1.1.0 available.


In [2]:
weights = 'bert-base-uncased'
tokenizer = transformers_modif.transformers.BertTokenizer.from_pretrained(weights)

model = modif_bert.BertForMaskedLM.from_pretrained(weights)
bert_model = model.bert
language_model = model.cls

I1118 16:43:35.521104 140578894108480 tokenization_utils.py:373] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/bertrand/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I1118 16:43:37.300052 140578894108480 configuration_utils.py:152] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/bertrand/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.bf3b9ea126d8c0001ee8a1e8b92229871d06d36d8808208cc2449280da87785c
I1118 16:43:37.303895 140578894108480 configuration_utils.py:169] Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": 

In [3]:
ids = tokenizer.encode("I was eating a sandwich.", add_special_tokens=True)
tok = [ tokenizer._convert_id_to_token(t) for t in ids ]
tok

['[CLS]', 'i', 'was', 'eating', 'a', 'sandwich', '.', '[SEP]']

In [33]:
def gen_sent(corpus):
    nlp = spacy.load("en_core_web_sm")
    for doc in corpus:
        for sent in nlp(doc).sents:
            yield list(zip(*[ [t.text, t.pos_, t.dep_] for t in sent ]))

def mask_unk(text):
    target_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    mask = [1] * len(target_ids[0])
    for i in range(1, len(target_ids[0]) - 1):
        if target_ids[i] == 100:   
            mask[i] = 0
    return mask

def encode_ids(text):
    target_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    tok = [ tokenizer._convert_id_to_token(t) for t in target_ids[0].numpy() ]
    input_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
    for index_target in range(1, len(input_ids[0]) - 1):
        if target_ids[0][index_target] == 100:
            continue
        input_ids[0][index_target] = 103
        yield input_ids, target_ids, index_target, tok
        input_ids[0][index_target] = target_ids[0][index_target]
    yield (), (), (), ()

In [5]:
KD_loss = nn.KLDivLoss(reduction='batchmean')
def loss_distrib(prediction_scores, teacher):
    input = prediction_scores[0][index_target].view(-1, prediction_scores.shape[2])
    student = F.log_softmax(input, dim=-1)
    return KD_loss(student, teacher)

loss = nn.CrossEntropyLoss()
def loss_cross_entropy(prediction_scores, target_ids, index_target):
    input = prediction_scores[0][index_target].view(-1, prediction_scores.shape[2])
    target = target_ids[0][index_target].view(-1)
    return loss(input, target)

def ptokens(prediction_scores):
    a = prediction_scores[0][index_target].view(-1, prediction_scores.shape[2])
    d = np.argsort(-a[0].detach().numpy())
    for i in d[:10]:
        print ("{:<20}{:<20}{}".format(tokenizer._convert_id_to_token(i), i, a[0][i]))
    print ()

In [20]:
def norm1(model):
    norm_l1 = 0
    for param in model.parameters():
        if param.requires_grad:
            norm_l1 += param.norm(p=1)
    return norm_l1

def clip_value(model):
    for param in model.parameters():
        if param.requires_grad:
            param.data.clamp_(min=0., max=10.)

def init_gates(model, lr):
    for param in model.parameters():
        param.requires_grad = False
    model.init_head_gates()
    return optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)

def get_target(target_ids):
    with torch.no_grad():
        outputs = bert_model(target_ids)
        prediction_scores = language_model(outputs[0])
        #target_scores = prediction_scores[0][index_target].view(-1, prediction_scores.shape[2])
        #teacher = F.softmax(target_scores, dim=-1)
        return prediction_scores

In [7]:
def gates_values(model):
    gates = list(filter(lambda p: p.requires_grad, model.parameters()))
    return np.array([ e.cpu().detach().numpy().flatten() for e in gates ])

def round_gates(gates):
    gates[gates < 0.5] = 0
    gates[gates > 0.5] = 1
    return gates

In [ ]:
corpus = open("../corpus.txt").readlines()
gen_s = gen_sent(corpus)
text, pos, dep = next(gen_s)
print (text)

In [ ]:
input_ids, target_ids, index_target, tokens = next(gen_ids)
tokens[index_target]

In [31]:
target = ["Here is the text to encode", "Next to me is the cat", "Generating sentences from a Continuous Space",
          "Wikipedia is a free online website, created and edited by volunteers around the world and hosted by the Wikimedia Foundation.",
          "She eats an apple", "An apple eats John",
          "i am for dialogue and i keep on saying this",
          "all four tasmanian boats have finished ."]

inp = ["Here is [MASK] text to encode",
       "Next to me is the [MASK]", 
       "Generating [MASK] from a continuous Space",
       "Wikipedia is a free online [MASK], created and edited by volunteers around the world and hosted by the Wikimedia Foundation.",
       "She [MASK] an apple",
       "An apple [MASK] John",
       "i am for [MASK] and i keep on saying this",
       "all four tasmanian [MASK] have finished ."]

sent = 6

input_ids = torch.tensor([tokenizer.encode(inp[sent], add_special_tokens=True)])
target_ids = torch.tensor([tokenizer.encode(target[sent], add_special_tokens=True)])

index_target = 4

# [MASK] = 103
input_ids, target_ids


(tensor([[ 101, 1045, 2572, 2005,  103, 1998, 1045, 2562, 2006, 3038, 2023,  102]]),
 tensor([[ 101, 1045, 2572, 2005, 7982, 1998, 1045, 2562, 2006, 3038, 2023,  102]]))

In [30]:
 tokenizer._convert_id_to_token(1012)

'.'

In [ ]:
[ 101,  101, 1045, 2572,  103, 1998, 1045, 2562, 2006, 3038, 2023,  102

In [ ]:
 101, 1045, 2572, 2005, 7982, 1998, 1045, 2562, 2006, 3038, 2023,  102

In [32]:
lr = 0.1
sigma = 0.1
epochs = 50

optimizer = init_gates(bert_model, lr)
target_scores = get_target(target_ids)

ptokens(target_scores)
ptokens(language_model(bert_model(input_ids)[0]))

for _ in range(epochs):
    optimizer.zero_grad()
    prediction_scores = language_model(bert_model(input_ids)[0])
    l = loss_cross_entropy(prediction_scores, target_ids, index_target)
    n = norm1(bert_model)

    print ("{:<40}{}".format(l.item(), n.item()))

    l += (sigma * n)
    l.backward()
    optimizer.step()
    clip_value(bert_model)

print ()
ptokens(prediction_scores)
gates = gates_values(bert_model)
print (sum(round_gates(gates.copy().flatten())))

dialogue            7982                18.581022262573242
dialogues           22580               12.793519973754883
debate              5981                10.235848426818848
reconciliation      16088               9.292250633239746
reflection          9185                8.949795722961426
unity               8499                8.89005184173584
clarity             15563               8.853912353515625
discussion          6594                8.655020713806152
understanding       4824                8.214696884155273
consensus           10465               7.789949417114258

you                 2017                7.599301815032959
real                2613                7.527132511138916
nothing             2498                6.66353702545166
sale                5096                6.581860065460205
him                 2032                6.0994367599487305
her                 2014                5.7508039474487305
sure                2469                5.581497669219971
them      

In [ ]:
import matplotlib.pyplot as plt
plt.matshow(gates)
plt.show()
plt.matshow(r_gates)
plt.show()

In [ ]:
r_gates

In [11]:
target_ids

tensor([[ 101, 1045, 2572, 2005, 7982, 1998, 1045, 2562, 2006, 3038, 2023,  102]])